In [7]:
#导入包
import numpy as np
import pandas as pd
from math import log
# import matplotlib.pyplot as plt

# %matplotlib inline

# from sklearn.datasets import load_iris
# from sklearn.model_selection import train_test_split

# from collections import Counter
# import math
# from math import log

# import pprint

In [2]:
# 数据集和分类属性

def create_data():
    datasets = [['青年', '否', '否', '一般', '否'],  # 数据集
               ['青年', '否', '否', '好', '否'],
               ['青年', '是', '否', '好', '是'],
               ['青年', '是', '是', '一般', '是'],
               ['青年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '好', '否'],
               ['中年', '是', '是', '好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '好', '是'],
               ['老年', '是', '否', '好', '是'],
               ['老年', '是', '否', '非常好', '是'],
               ['老年', '否', '否', '一般', '否'],]

    labels = [u'年龄', u'有工作', u'有自己的房子', u'信贷情况', u'类别']  # 分类属性

    return datasets, labels  # 返回数据集和分类属性

In [3]:
# 将数据集转为DataFrame表格

datasets, labels = create_data()
train_data = pd.DataFrame(datasets, columns=labels)
train_data

,年龄,有工作,有自己的房子,信贷情况,类别
0,青年,否,否,一般,否
1,青年,否,否,好,否
2,青年,是,否,好,是
3,青年,是,是,一般,是
4,青年,否,否,一般,否
5,中年,否,否,一般,否
6,中年,否,否,好,否
7,中年,是,是,好,是
8,中年,否,是,非常好,是
9,中年,否,是,非常好,是


In [4]:
def calc_ent(datasets):
    data_length = len(datasets)
    label_count = {}
    for i in range(data_length):
        label = datasets[i][-1]
        if label not in label_count:
            label_count[label] = 0
        label_count[label] += 1
    ent = -sum([(p/data_length)*log(p/data_length, 2) for p in label_count.values()])
    return ent

In [8]:
ent = calc_ent(datasets)
ent

0.9709505944546686

In [9]:
def cond_ent(datasets, axis=0):
    data_length = len(datasets)
    feature_sets = {}
    for i in range(data_length):
        feature = datasets[i][axis]
        if feature not in feature_sets:
            feature_sets[feature] = []
        feature_sets[feature].append(datasets[i])
    cond_ent = sum([(len(p)/data_length)*calc_ent(p) for p in feature_sets.values()])
    return cond_ent

In [10]:
cond_ent1= cond_ent(datasets)
cond_ent1

0.8879430945988998

In [11]:
def info_gain(ent, cond_ent):
    return ent - cond_ent

def info_gain_train(datasets):
    count = len(datasets[0]) - 1
    ent = calc_ent(datasets)
    best_feature = []
    for c in range(count):
        c_info_gain = info_gain(ent, cond_ent(datasets, axis=c))
        best_feature.append((c, c_info_gain))
        print("特征(%s)的信息增益位:%.3f" % (labels[c], c_info_gain))
    # 比较大小
    best_ = max(best_feature, key=lambda x: x[-1])
    return "特征%s的信息增益最大,选择为根节点特征" % (labels[best_[0]])

In [12]:
info_gain_train(np.array(datasets))

特征(年龄)的信息增益位:0.083
特征(有工作)的信息增益位:0.324
特征(有自己的房子)的信息增益位:0.420
特征(信贷情况)的信息增益位:0.363


'特征有自己的房子的信息增益最大,选择为根节点特征'

In [13]:
##利用ID3算法生成决策树，例5.3
# 定义节点类 二叉树
class Node:
    def __init__(self, root=True, label=None, feature_name=None, feature=None):
        self.root = root
        self.label = label
        self.feature_name = feature_name
        self.feature = feature
        self.tree = {}
        self.result = {'label:': self.label, 'feature': self.feature, 'tree': self.tree}

    def __repr__(self):
        return '{}'.format(self.result)

    def add_node(self, val, node):
        self.tree[val] = node

    def predict(self, features):
        if self.root is True:
            return self.label
        return self.tree[features[self.feature]].predict(features)
    
class DTree:
    def __init__(self, epsilon=0.1):
        self.epsilon = epsilon
        self._tree = {}

    # 熵
    @staticmethod
    def calc_ent(datasets):
        data_length = len(datasets)
        label_count = {}
        for i in range(data_length):
            label = datasets[i][-1]
            if label not in label_count:
                label_count[label] = 0
            label_count[label] += 1
        ent = -sum([(p/data_length)*log(p/data_length, 2) for p in label_count.values()])
        return ent

    # 经验条件熵
    def cond_ent(self, datasets, axis=0):
        data_length = len(datasets)
        feature_sets = {}
        for i in range(data_length):
            feature = datasets[i][axis]
            if feature not in feature_sets:
                feature_sets[feature] = []
            feature_sets[feature].append(datasets[i])
        cond_ent = sum([(len(p)/data_length)*self.calc_ent(p) for p in feature_sets.values()])
        return cond_ent

    # 信息增益
    @staticmethod
    def info_gain(ent, cond_ent):
        return ent - cond_ent

    def info_gain_train(self, datasets):
        count = len(datasets[0]) - 1
        ent = self.calc_ent(datasets)
        best_feature = []
        for c in range(count):
            c_info_gain = self.info_gain(ent, self.cond_ent(datasets, axis=c))
            best_feature.append((c, c_info_gain))
        # 比较大小
        best_ = max(best_feature, key=lambda x: x[-1])
        return best_

    def train(self, train_data):
        """
        input:数据集D(DataFrame格式)，特征集A，阈值eta
        output:决策树T
        """
        _, y_train, features = train_data.iloc[:, :-1], train_data.iloc[:, -1], train_data.columns[:-1]
        # 1,若D中实例属于同一类Ck，则T为单节点树，并将类Ck作为结点的类标记，返回T
        if len(y_train.value_counts()) == 1:
            return Node(root=True,
                        label=y_train.iloc[0])

        # 2, 若A为空，则T为单节点树，将D中实例树最大的类Ck作为该节点的类标记，返回T
        if len(features) == 0:
            return Node(root=True, label=y_train.value_counts().sort_values(ascending=False).index[0])

        # 3,计算最大信息增益 同5.1,Ag为信息增益最大的特征
        max_feature, max_info_gain = self.info_gain_train(np.array(train_data))
        max_feature_name = features[max_feature]

        # 4,Ag的信息增益小于阈值eta,则置T为单节点树，并将D中是实例数最大的类Ck作为该节点的类标记，返回T
        if max_info_gain < self.epsilon:
            return Node(root=True, label=y_train.value_counts().sort_values(ascending=False).index[0])

        # 5,构建Ag子集
        node_tree = Node(root=False, feature_name=max_feature_name, feature=max_feature)

        feature_list = train_data[max_feature_name].value_counts().index
        for f in feature_list:
            sub_train_df = train_data.loc[train_data[max_feature_name] == f].drop([max_feature_name], axis=1)

            # 6, 递归生成树
            sub_tree = self.train(sub_train_df)
            node_tree.add_node(f, sub_tree)

        # pprint.pprint(node_tree.tree)
        return node_tree

    def fit(self, train_data):
        self._tree = self.train(train_data)
        return self._tree

    def predict(self, X_test):
        return self._tree.predict(X_test)

In [14]:
datasets, labels = create_data()
data_df = pd.DataFrame(datasets, columns=labels)
dt = DTree()
tree = dt.fit(data_df)
tree

{'label:': None, 'feature': 2, 'tree': {'否': {'label:': None, 'feature': 1, 'tree': {'否': {'label:': '否', 'feature': None, 'tree': {}}, '是': {'label:': '是', 'feature': None, 'tree': {}}}}, '是': {'label:': '是', 'feature': None, 'tree': {}}}}

In [14]:
2

2

In [15]:
## 导包

# 数据集
from sklearn import datasets
# 分类器
from sklearn import tree
# 训练集测试集分割模块
from sklearn.model_selection import train_test_split
# 绘制决策树
import graphviz

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [16]:
# 自定义导入数据集函数
def get_data(total_data):
    # 显示total_data包含的内容
    print("传入数据集包含内容有：", [x for x in total_data.keys()])
    # 样本
    x_true = total_data.data
    # 标签
    y_true = total_data.target
    # 特征名称
    feature_names = total_data.feature_names
    # 类名
    target_names = total_data.target_names
    
    return x_true, y_true, feature_names, target_names

In [17]:
# 定义主函数
def main():
    # 利用自定义函数导入Iris数据集
    total_iris = datasets.load_iris()
    x_true, y_true, feature_names, target_names = get_data(total_iris)
    
    # 分割数据集
    rate_test = 0.2  # 训练集比例
    x_train, x_test, y_train, y_test = train_test_split(x_true,
                                                        y_true,
                                                        test_size= rate_test)
    print("\n训练集样本大小：", x_train.shape)
    print("训练集标签大小：", y_train.shape)
    print("测试集样本大小：", x_test.shape)
    print("测试集标签大小：", y_test.shape)

    # 设置决策树分类器
    clf = tree.DecisionTreeClassifier(criterion="entropy")
    # 训练模型
    clf.fit(x_train, y_train)
    # 评价模型
    score = clf.score(x_test, y_test)
    print("\n模型测试集准确率为：", score)
    
    # 绘制决策树模型
    clf_dot = tree.export_graphviz(clf,
                                   out_file= None,
                                   feature_names= feature_names,
                                   class_names= target_names,
                                   filled= True,
                                   rounded= True)
    # 显示绘制的模型，在当前目录下，保存为png模式
    graph = graphviz.Source(clf_dot, 
    						filename= "iris_decisionTree.gv", 
    						format= "png")
    graph.view()
    
    # 显示特征重要程度
    print("\n特征重要程度为：")
    info = [*zip(feature_names, clf.feature_importances_)]
    for cell in info:
        print(cell)
    
    
# 调用主函数
if __name__ == "__main__":
    main()

传入数据集包含内容有： ['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module']

训练集样本大小： (120, 4)
训练集标签大小： (120,)
测试集样本大小： (30, 4)
测试集标签大小： (30,)

模型测试集准确率为： 0.9333333333333333

特征重要程度为：
('sepal length (cm)', 0.05127253911942476)
('sepal width (cm)', 0.03420743397910834)
('petal length (cm)', 0.29553432725251055)
('petal width (cm)', 0.6189856996489563)


In [18]:
import pandas as pd
from sklearn.datasets import load_iris
iris=load_iris()
x=iris.data
y=iris.target
data=pd.DataFrame(x,columns=['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm'])
labels=pd.DataFrame(y,columns=['Species'])
data

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [19]:
dt = DTree()
tree = dt.fit(data)
tree

{'label:': None, 'feature': 2, 'tree': {1.4: {'label:': None, 'feature': 0, 'tree': {5.1: {'label:': 0.2, 'feature': None, 'tree': {}}, 4.9: {'label:': None, 'feature': 0, 'tree': {3.0: {'label:': 0.2, 'feature': None, 'tree': {}}, 3.6: {'label:': 0.1, 'feature': None, 'tree': {}}}}, 5.0: {'label:': 0.2, 'feature': None, 'tree': {}}, 4.6: {'label:': None, 'feature': 0, 'tree': {3.4: {'label:': 0.3, 'feature': None, 'tree': {}}, 3.2: {'label:': 0.2, 'feature': None, 'tree': {}}}}, 4.8: {'label:': 0.1, 'feature': None, 'tree': {}}, 4.4: {'label:': 0.2, 'feature': None, 'tree': {}}, 5.2: {'label:': 0.2, 'feature': None, 'tree': {}}, 5.5: {'label:': 0.2, 'feature': None, 'tree': {}}}}, 1.5: {'label:': None, 'feature': 1, 'tree': {3.1: {'label:': None, 'feature': 0, 'tree': {4.9: {'label:': 0.1, 'feature': None, 'tree': {}}, 4.6: {'label:': 0.2, 'feature': None, 'tree': {}}}}, 3.4: {'label:': None, 'feature': 0, 'tree': {5.0: {'label:': 0.2, 'feature': None, 'tree': {}}, 5.4: {'label:': 0.4